# Configurational Space Partitioner

In the interest of partitioning the configuration space of small molecules (in different phases), it is necessary to take some dataset of small molecule configurations (hopefully boltzmann distributed) and perform some partitioning.

## Featurizer

I am going to postulate that heavy torsion terms are sufficient to partition the configurational space accordingly. Let's try that...<br>
I've been told that [pyEmma](https://github.com/markovmodel/PyEMMA) can perform featurization without much pain...

First, I'll load the trajectory into mdtraj

In [43]:
import mdtraj as md
import numpy as np
import pyemma
from openforcefield.topology import Molecule
import openeye.oechem as oechem

In [67]:
traj = md.Trajectory.load('lig0.solvent.forward.pdb')
topology = traj.topology
openmm_topology = topology.to_openmm()
off_mol = Molecule.from_file('lig0.solvent.forward.pdb')
template_oemol = off_mol[0].to_openeye()
traj.save_xtc('lig0.solvent.forward.xtc') #wasn't saved in a pyemma-safe format

In [64]:
off_mol[0]._conformers

[Quantity(value=array([[ 8.4659996 ,  5.26000023,  4.27199984],
        [ 8.99199963,  5.93599987,  3.24699998],
        [ 8.4090004 ,  5.89499998,  1.94200003],
        [ 7.25899982,  5.10900021,  1.745     ],
        [ 6.81599998,  4.2750001 ,  2.82200003],
        [ 7.36600018,  4.4000001 ,  4.1329999 ],
        [ 5.43599987,  3.20799994,  2.69899988],
        [ 6.54099989,  5.204     ,  0.44299999],
        [ 6.15600014,  4.2249999 , -0.09      ],
        [ 6.25500011,  6.51200008,  0.014     ],
        [ 5.35500002,  6.796     , -1.03199995],
        [ 5.0630002 ,  5.91599989, -2.08299994],
        [ 4.28299999,  6.34299994, -3.13599992],
        [ 3.60700011,  7.5       , -3.20300007],
        [ 3.93799996,  8.35400009, -2.2019999 ],
        [ 4.78999996,  8.03800011, -1.11600006],
        [ 3.23000002,  9.51599979, -2.24099994],
        [ 3.22900009, 10.53899956, -1.33200002],
        [ 3.88800001, 10.56200027, -0.28400001],
        [ 2.27800012, 11.7130003 , -1.60800004],
     

In [46]:
template_oemol

<openeye.oechem.OEMol; proxy of <Swig Object of type '_p_OEMolWrapper' at 0x7fcaf94366c0> >

In [53]:
class IsRotatableOrMacroCycleBond(oechem.OEUnaryBondPred):
    """
    Identifies rotatable bonds and single bonds in macro-cycles.
    """
    def __call__(self, bond):
        """
        :type mol: oechem.OEBondBase
        :rtype: boolean
        """
        if bond.GetOrder() != 1:
            return False
        if bond.IsAromatic():
            return False

        isrotor = oechem.OEIsRotor()
        if isrotor(bond):
            return True

        if oechem.OEBondGetSmallestRingSize(bond) >= 10:
            return True

        return False
def get_dihedrals(mol, itag):
    """
    Iterates over rotatable bonds and identifies their dihedral
    atoms. These atoms are added to the molecule in a group
    using the given tag.

    :type mol: oechem.OEMol
    :type itag: int
    :return: Number of dihedral angles identified
    :rtype: int
    """
    nrdihedrals = 0
    for bond in mol.GetBonds(IsRotatableOrMacroCycleBond()):
        atomB = bond.GetBgn()
        atomE = bond.GetEnd()

        neighB = None
        neighE = None

        for atom in atomB.GetAtoms(oechem.OEIsHeavy()):
            if atom != atomE:
                neighB = atom
                break
        for atom in atomE.GetAtoms(oechem.OEIsHeavy()):
            if atom != atomB:
                neighE = atom
                break

        if neighB is None or neighE is None:
            continue

        atomorder = [neighB, atomB, atomE, neighE]
        bondorder = [mol.GetBond(neighB, atomB), bond, mol.GetBond(neighE, atomE)]

        if neighB.GetIdx() < neighE.GetIdx():
            atomorder.reverse()
            bondorder.reverse()

        atoms = oechem.OEAtomVector(atomorder)
        bonds = oechem.OEBondVector(bondorder)

        nrdihedrals += 1
        mol.NewGroup(itag, atoms, bonds)

    return nrdihedrals

In [54]:
get_dihedrals(mol = template_oemol, itag=0)

5

In [59]:
mol = template_oemol
itag=0

In [63]:
for group in mol.GetGroups(oechem.OEHasGroupType(itag)):
    for atom in group.GetAtoms():
        print(atom.GetIdx())
    print()
    

17
16
14
13

19
17
16
14

11
10
9
7

10
9
7
8

8
7
3
2



<openeye.oechem.OEAtomVector; proxy of <Swig Object of type 'std::vector< OEChem::OEAtomBase * > *' at 0x7fcaf35cde70> >